In [11]:
import string
import re
from datetime import datetime
import locale
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [12]:
pd.set_option('display.max_colwidth', 500)

# Global Functions and Variable

In [13]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

stop_words = set(stopwords.words('indonesian'))
exceptions = {"sampai", "kurang", "lebih", "dari", "hari"}
custom_stop_words = stop_words - exceptions

In [14]:
istilah_kesehatan = {
    # Tekanan Darah
    "hipertensi": "tekanan darah tinggi",
    "hipotensi": "tekanan darah rendah",

    # Gula Darah
    "hiperglikemia": "kadar gula darah tinggi",
    "hipoglikemia": "kadar gula darah rendah",
    "diabetes melitus": "penyakit kencing manis",

    # Pernapasan
    "dispnea": "sesak napas",
    "rinorea": "pilek atau ingusan",
    "epistaksis": "mimisan",
    "faringitis": "radang tenggorokan",
    "laringitis": "radang pita suara",
    "asma": "bengek",

    # Pencernaan
    "dispepsia": "mag atau gangguan pencernaan",
    "konstipasi": "sembelit atau susah buang air besar",
    "diare": "mencret",
    "hemoroid": "wasir atau ambeien",
    "apendisitis": "radang usus buntu",
    "gastritis": "radang lambung",

    # Kulit dan Alergi
    "urtikaria": "biduran atau kaligata",
    "dermatitis": "eksim atau radang kulit",
    "varisela": "cacar air",
    "morbili": "campak atau tampek",
    "veruka": "kutil",
    "tinea pedis": "kutu air",
    "lotion": "losion",

    # Kepala dan Saraf
    "sefalgia": "sakit kepala atau pusing",
    "insomnia": "susah tidur",
    "sinkop": "pingsan",
    "konvulsi": "kejang",

    # Umum
    "pireksia": "demam",
    "hipertermia": "suhu tubuh sangat tinggi",
    "hipotermia": "suhu tubuh sangat rendah",
    "mialgia": "nyeri otot",
    "artralgia": "nyeri sendi",
    "fatik": "kelelahan atau rasa capai",
    "edema": "bengkak",
    "pruritus": "gatal-gatal",
    "anemia": "kurang darah",
    "karsinoma": "kanker",
    "neoplasma": "tumor",
    "halitosis": "bau mulut",
    "kalkulus renal": "batu ginjal",
    "moisturizing": "moisturizer"
}

In [15]:
# Mapping bulan
bulan = {
    # Nama singkat
    "Jan": "Januari",
    "Feb": "Februari",
    "Mar": "Maret",
    "Apr": "April",
    "Mei": "Mei",
    "Jun": "Juni",
    "Jul": "Juli",
    "Agu": "Agustus",
    "Sep": "September",
    "Okt": "Oktober",
    "Nov": "November",
    "Des": "Desember",

    # Angka
    "01": "Januari",
    "02": "Februari",
    "03": "Maret",
    "04": "April",
    "05": "Mei",
    "06": "Juni",
    "07": "Juli",
    "08": "Agustus",
    "09": "September",
    "10": "Oktober",
    "11": "November",
    "12": "Desember",
}

In [16]:
# Fungsi untuk stemming
def stem_tokens(tokens):
    lemmas = [stemmer.stem(token) for token in tokens]
    return lemmas

In [17]:
def normalize_text(text, dictionary):
    text = re.sub(r'(\d+)\s*[–-]\s*(\d+)', r'\1 sampai \2', text)
    text = text.replace('>', ' lebih dari ')
    text = text.replace('<', ' kurang dari ')
    
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [18]:
def preprocess_title_desc(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

# Biofarma

In [19]:
biofarma_df = pd.read_csv('../artikel/biofarma.csv')
biofarma_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        108 non-null    object 
 1   tag          0 non-null      float64
 2   link         108 non-null    object 
 3   date         108 non-null    object 
 4   description  108 non-null    object 
 5   content      108 non-null    object 
dtypes: float64(1), object(5)
memory usage: 5.2+ KB


In [20]:
biofarma_df['source'] = 'Biofarma'
biofarma_df.head()

,title,tag,link,date,description,content,source
0,5 Cara Merawat Ginjal agar Sehat & Cegah Penyakit Ginjal,NaN,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,"Rabu, 3 September 2025",Kenali cara merawat ginjal agar terhindar dari gejala ginjal bermasalah. Pelajari ciri penyakit ginjal dan pencegahan penyakit ginjal apa saja sejak dini.,"Ginjal adalah organ vital yang berfungsi menyaring darah, membuang limbah, dan menjaga keseimbangan cairan tubuh. Jika tidak dijaga, fungsi ginjal bisa menurun dan menimbulkan masalah serius. Banyak penyakit ginjal apa saja yang berkembang tanpa gejala pada tahap awal, sehingga penting menjaga kesehatan ginjal sejak dini.\nBerikut ini adalah 5 cara merawat ginjal yang sederhana namun efektif untuk mencegah gejala ginjal bermasalah:\n1. Minum Air Putih yang Cukup\nAsupan cairan yang cukup mem...",Biofarma
1,7 Tips Untuk Menjaga Kesehatan Mata,NaN,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,"Jumat, 29 Agustus 2025","Tips menjaga kesehatan mata: nutrisi bergizi, kacamata pelindung, olahraga, hingga periksa rutin ke dokter","Menjaga kesehatan mata sangatlah penting karena mata berperan besar dalam hampir semua aktivitas sehari-hari, mulai dari belajar, bekerja, hingga berinteraksi dengan lingkungan sekitar. Jika kesehatan mata terabaikan, berbagai masalah bisa muncul, mulai dari iritasi ringan, infeksi, hingga penyakit serius yang berisiko menyebabkan kebutaan. Untuk itu, menjaga mata dapat dilakukan melalui pemeriksaan rutin ke dokter dan juga dengan membiasakan pola hidup sehat yang mendukung fungsi penglihata...",Biofarma
2,Jangan Sepelekan! 10 Penyebab Campak yang Sering Terabaikan,NaN,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,"Kamis, 28 Agustus 2025","Kenali faktor utama penyebab campak, gejala khasnya, dan cara pencegahan melalui imunisasi MMR","Penyebab Campak\nCampak adalah penyakit menular yang disebabkan oleh virus Morbili dan dapat menyebar dengan sangat cepat dari satu orang ke orang lain. Mekanisme utama penularannya adalah melalui percikan air liur (droplet) yang keluar dari mulut atau hidung penderita saat batuk, bersin, atau berbicara. Selain itu, seseorang juga dapat tertular jika menyentuh benda yang sudah terkontaminasi virus lalu menyentuh hidung, mulut, atau mata. Berikut adalah sepuluh faktor penyebab yang paling ber...",Biofarma
3,"Campak - Penyebab, Gejala, dan Pencegahannya",NaN,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,"Senin, 25 Agustus 2025",Campak bisa berbahaya bila dibiarkan. Periksa ke dokter sejak gejala awal muncul agar penanganan lebih tepat.,"Apa itu campak?\nCampak atau rubella adalah penyakit menular yang disebabkan oleh virus dari keluarga Paramyxoviridae dengan genus Morbillivirus. Penyakit ini menyerang saluran pernapasan terlebih dahulu, lalu menimbulkan gejala sistemik seperti demam, batuk, pilek, sakit tenggorokan, mata merah, hingga ruam menyeluruh di kulit. Campak termasuk penyakit yang sangat menular, bisa berpindah dari orang ke orang hanya melalui udara, percikan air liur, atau benda yang terkontaminasi virus.\nApa s...",Biofarma
4,Cara Mudah Menjaga Kesehatan Gigi dan Mulut agar Terhindar dari Penyakit,NaN,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,"Rabu, 20 Agustus 2025","Kesehatan gigi dan mulut adalah kondisi ketika gigi, gusi, lidah, dan seluruh rongga mulut terjaga kebersihan serta fungsinya dengan baik. Mulut merupakan gerbang utama bagi makanan dan minuman yang masuk ke dalam tubuh, sekaligus organ yang berperan...","Apa itu Kesehatan Gigi dan Mulut?\nKesehatan gigi dan mulut adalah kondisi ketika gigi, gusi, lidah, dan seluruh rongga mulut terjaga kebersihan serta fungsinya dengan baik. Mulut merupakan gerbang utama bagi makanan dan minuman yang masuk

In [21]:
biofarma_df.isnull().sum()

title            0
tag            108
link             0
date             0
description      0
content          0
source           0
dtype: int64

In [22]:
biofarma_df['tag'] = biofarma_df['tag'].fillna('Kesehatan Umum')

In [23]:
biofarma_df['title'].duplicated().sum()

np.int64(0)

In [24]:
def preprocess_date_biofarma(text):
    text = re.sub(r'^\s*\w+,\s*', '', text)
    return text.strip()

In [25]:
def preprocess_content_biofarma(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = re.sub(r'referensi[\s\S]*?(bagikan:|download|$)', '', text, flags=re.IGNORECASE) # Remove blok referensi ke akhir
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [26]:
biofarma_df['title'] = biofarma_df['title'].apply(lambda x: preprocess_title_desc(x))
biofarma_df['description'] = biofarma_df['description'].apply(lambda x: preprocess_title_desc(x))
biofarma_df['content'] = biofarma_df['content'].apply(lambda x: preprocess_content_biofarma(x))
biofarma_df['date'] = biofarma_df['date'].apply(lambda x: preprocess_date_biofarma(x))

In [27]:
biofarma_df.head()

,title,tag,link,date,description,content,source
0,5 rawat ginjal sehat cegah sakit ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,3 September 2025,kenal rawat ginjal hindar dari gejala ginjal masalah ajar ciri sakit ginjal cegah sakit ginjal,ginjal organ vital fungsi nyaring darah buang limbah jaga imbang cair tubuh jaga fungsi ginjal turun timbul serius sakit ginjal kembang gejala tahap jaga sehat ginjal 5 rawat ginjal sederhana efektif cegah gejala ginjal masalah 1 minum air putih asupan cair bantu ginjal optimal buang racun sisa metabolisme kurang minum air sebab dehidrasi tingkat risiko batu ginjal biasa minum 8 sampai 10 gelas air putih hari jaga fungsi ginjal sehat 2 batas konsumsi garam asupan garam lebih tingkat tekan da...,Biofarma
1,7 tips jaga sehat mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,29 Agustus 2025,tips jaga sehat mata nutrisi gizi kacamata lindung olahraga periksa rutin dokter,jaga sehat mata mata peran aktivitas seharihari dari ajar interaksi lingkung sehat mata abai muncul dari iritasi ringan infeksi sakit serius risiko sebab buta jaga mata periksa rutin dokter biasa pola hidup sehat dukung fungsi lihat optimal iring tambah usia biasa seharihari kurang sehat fungsi mata turun cegah langkah sederhana jaga asupan nutrisi istirahat tatap layar lindung mata dari papar sinar lebih jaga kualitas lihat tahan lebih tips jaga sehat mata mata salah organ aktif hari lihat ...,Biofarma
2,sepele 10 sebab campak abai,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,28 Agustus 2025,kenal faktor utama sebab campak gejala khas cegah imunisasi mmr,sebab campak campak sakit tular sebab virus campak tampek sebar cepat dari orang orang mekanisme utama tular percik air liur droplet dari mulut hidung derita batuk bersin bicara tular sentuh benda kontaminasi virus sentuh hidung mulut mata puluh faktor sebab pengaruh vaksinasi campak mmr campak serang individu vaksin vaksin mmr measles mumps rubella bukti efektif cegah sakit anakanak terima vaksin lengkap milik risiko lebih infeksi milik kebal tubuh virus campak tampek virus usia bayi muda v...,Biofarma
3,campak sebab gejala cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,25 Agustus 2025,campak bahaya biar periksa dokter gejala muncul tangan lebih,campak campak rubella sakit tular sebab virus dari keluarga paramyxoviridae genus morbillivirus sakit serang salur napas timbul gejala sistemik demam batuk pilek sakit tenggorok mata merah ruam kulit campak sakit tular pindah dari orang orang udara percik air liur benda kontaminasi virus gejala campak gejala campak muncul inkubasi 7 sampai 14 hari gejala flu demam lemas batuk kering pilek sakit tenggorok diare muntah mata merah air sensitif cahaya bercak putih bintik koplik muncul mulut hari...,Biofarma
4,mudah jaga sehat gigi mulut hindar dari sakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,20 Agustus 2025,sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran,sehat gigi mulut sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran proses cerna makan masuk tinggal mulut bersih sisa kembang bakteri sehat mulut kait indah senyum napas segar tentu sehat tubuh akibat jaga sehat gigi mulut abai bersih mulut gigi picu ganggu sehat muncul bau mulut halitosis sisa makan busuk mulut akibat biasa rokok gigi lubang akibat tumpuk plak kikis email gigi radang gusi gingivitis atas kembang ...,Biofarma


# Klikdokter

In [28]:
klikdokter_df = pd.read_csv('../artikel/klikdokter.csv')
klikdokter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        110 non-null    object
 1   tag          110 non-null    object
 2   link         110 non-null    object
 3   date         110 non-null    object
 4   description  103 non-null    object
 5   content      110 non-null    object
dtypes: object(6)
memory usage: 5.3+ KB


In [29]:
klikdokter_df['source'] = 'Klikdokter'
klikdokter_df.head()

,title,tag,link,date,description,content,source
0,Jangan Panik! Kenali Rabies dan Pertolongan Pertama Saat Digigit Hewan Liar,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/pertolongan-pertama-saat-digigit-hewan-liar,30 Mei 2025,NaN,"Berita Kesehatan\nJangan Panik! Kenali Rabies dan Pertolongan Pertama Saat Digigit Hewan Liar\ndr. Dyah Novita, 30 Mei 2025\nDitinjau Oleh Tim Medis Klikdokter\nBagikan\nTahukah kamu bahwa rabies sangat berbahaya dan dapat mengancam nyawa? Penyakit ini menyerang sistem saraf pusat dan dapat menular melalui gigitan hewan yang terinfeksi, terutama anjing. Tapi jangan khawatir, dengan mengenal rabies, gejalanya, dan cara pencegahannya, Kamu bisa menjaga diri dan orang di sekitarmu tetap aman. ...",Klikdokter
1,Terapi Stem Cell untuk Pengobatan Penyakit Autoimun,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/terapi-stem-cell-untuk-pengobatan-penyakit-autoimun,29 Apr 2025,Penyakit autoimun seperti rheumatoid arthritis bisa menurunkan kualitas hidup. Terapi stem cell hadir sebagai harapan baru dalam pengobatannya.,"Berita Kesehatan\nTerapi Stem Cell untuk Pengobatan Penyakit Autoimun\ndr. Dyah Novita Anggraini, CBS, CIMI, 29 Apr 2025\nDitinjau oleh dr. Dyah Novita Anggraini, CBS, CIMI\nBagikan\nPenyakit autoimun seperti rheumatoid arthritis bisa menurunkan kualitas hidup. Terapi stem cell hadir sebagai harapan baru dalam pengobatannya.\n\nPenyakit autoimun, seperti rheumatoid arthritis menyebabkan peradangan pada sendi yang bisa menurunkan kualitas hidup. Umumnya, masalah kesehatan ini dikelola dengan ...",Klikdokter
2,"Terapi Stem Cell, Harapan Baru untuk Pengobatan Penyakit Degeneratif",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-pengobatan-penyakit-degeneratif-di-masa-depan,29 Apr 2025,Terapi stem cell kini menjadi harapan baru dalam menangani penyakit degeneratif akibat proses penuaan alami. Simak penjelasan lengkapnya di sini.,"Berita Kesehatan\nTerapi Stem Cell, Harapan Baru untuk Pengobatan Penyakit Degeneratif\ndr. Dyah Novita Anggraini, CBS, CIMI, 29 Apr 2025\nDitinjau oleh dr. Dyah Novita Anggraini, CBS, CIMI\nBagikan\nTerapi stem cell kini menjadi harapan baru dalam menangani penyakit degeneratif akibat proses penuaan alami. Simak penjelasan lengkapnya di sini.\nSemakin bertambah usia, penuaan akan terjadi secara alami. Tidak hanya kulit, organ penting di dalam tubuh seperti jantung, tulang, atau otak juga ak...",Klikdokter
3,"Stem Cell Tumbuhan dan Manusia Itu Berbeda, Ini Penjelasannya!",Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-tumbuhan-dan-manusia-itu-berbeda-ini-penjelasannya,19 Apr 2025,"Baik tumbuhan maupun manusia memiliki stem cell., Nnamun keduanya memiliki perbedaan dalam struktur, fungsi, dan aplikasinya. Penasaran? Yuk, pelajari lewat ulasan berikut!","Berita Kesehatan\nStem Cell Tumbuhan dan Manusia Itu Berbeda, Ini Penjelasannya!\ndr. Dyah Novita Anggraini, CBS, CIMI, 19 Apr 2025\nDitinjau oleh dr. Dyah Novita Anggraini, CBS, CIMI\nBagikan\nBaik tumbuhan maupun manusia memiliki stem cell., Nnamun keduanya memiliki perbedaan dalam struktur, fungsi, dan aplikasinya. Penasaran? Yuk, pelajari lewat ulasan berikut!\nApa Itu Stem Cell?\nSecara umum stem cell adalah sel yang belum memiliki fungsi khusus, namun dapat membelah diri dan nantinya b...",Klikdokter
4,Keberagaman Model di Victoria’s Secret Fashion Show dan Mengenal Bentuk Tubuh Perempuan,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/mengenal-keberagaman-bentuk-tubuh-perempuan,17 Okt 2024,"Victoria's Secret ubah standar kecantikan! Show terbaru ini merayakan keberagaman bentuk tubuh, warna kulit, dan ukuran. Sebuah pesan kuat tentang self-love dan body positivity.","Berita Kesehatan\nKeberagaman Model di Victoria’s Secret Fashion Show dan Mengenal Bentuk Tubuh Perempuan\nChristovel Ramot, 17 Okt 2024\nDitinjau oleh dr. Dyah Novita Anggraini\nBagikan\nVictoria's Secret ubah s

In [30]:
klikdokter_df.isnull().sum()

title          0
tag            0
link           0
date           0
description    7
content        0
source         0
dtype: int64

In [31]:
klikdokter_df['title'].duplicated().sum()

np.int64(0)

In [32]:
klikdokter_df['description'] = klikdokter_df['description'].fillna('Tidak ada deskripsi')

In [33]:
def preprocess_date_klikdokter(text, dictionary):    
    tokens = text.split()

    normalized_tokens = []
    for token in tokens:
        normalized_token = dictionary.get(token, token)
        normalized_tokens.append(normalized_token)

    return " ".join(normalized_tokens)

In [34]:
def preprocess_content_klikdokter(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'^.*?Bagikan', '', text, flags=re.DOTALL|re.IGNORECASE) # Remove pendahuluan artikel
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'^artikel lainnya:.*', '', text, flags=re.MULTILINE) # Remove artikel lainnya
    text = re.sub(r'Yuk, download.*?Referensi', '', text, flags=re.DOTALL|re.IGNORECASE) # Remove penutup artikel
    text = re.sub(r'Referensi.*', '', text, flags=re.IGNORECASE) # Remove referensi
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [35]:
klikdokter_df['title'] = klikdokter_df['title'].apply(lambda x: preprocess_title_desc(x))
klikdokter_df['description'] = klikdokter_df['description'].apply(lambda x: preprocess_title_desc(x))
klikdokter_df['content'] = klikdokter_df['content'].apply(lambda x: preprocess_content_klikdokter(x))
klikdokter_df['date'] = klikdokter_df['date'].apply(lambda x: preprocess_date_klikdokter(x, bulan))

In [36]:
klikdokter_df.head()

,title,tag,link,date,description,content,source
0,panik kenal rabies tolong gigit hewan liar,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/pertolongan-pertama-saat-digigit-hewan-liar,30 Mei 2025,deskripsi,tahu rabies bahaya ancam nyawa sakit serang sistem saraf pusat tular gigit hewan infeksi anjing khawatir kenal rabies gejala cegah jaga orang sekitar aman mari kenal rabies bahaya rabies rabies sakit infeksi serius serang sistem saraf pusat otak akibat lyssavirus sakit bahaya tular manusia gigit cakar hewan infeksi virus gigit hewan bawa virus rabies virus rabies masuk tubuh sebab infeksi potensi fatal mati hewan tular rabies manusia anjing kucing kera hewan liar rubah musang anjing liar sum...,Klikdokter
1,terapi stem cell obat sakit autoimun,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/terapi-stem-cell-untuk-pengobatan-penyakit-autoimun,29 April 2025,sakit autoimun rheumatoid arthritis turun kualitas hidup terapi stem cell hadir harap obat,sakit autoimun rheumatoid arthritis turun kualitas hidup terapi stem cell hadir harap obat sakit autoimun rheumatoid arthritis sebab adang sendi turun kualitas hidup sehat kelola konsumsi obat antiradang maju teknologi sehat stem cell yakin potensi salah obat sakit autoimun yuk simak ulas lengkap kenal stem cell sakit autoimun stem cell sel punca sel milik istimewa kembang jenis sel tubuh sel milik sifat utama selfrenewal belah banyak diferensiasi differentiation mampu stem cell kembang biak...,Klikdokter
2,terapi stem cell harap obat sakit degeneratif,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-pengobatan-penyakit-degeneratif-di-masa-depan,29 April 2025,terapi stem cell harap tangan sakit degeneratif akibat proses tua alami simak jelas lengkap,terapi stem cell harap tangan sakit degeneratif akibat proses tua alami simak jelas lengkap tambah usia tua alami kulit organ tubuh jantung tulang otak alami turun fungsi baik dari kondisi sebab sakit degeneratif teliti terapi stem cell potensi obat sakit degeneratif penasaran yuk simak ulas kenal terapi stem cell iring maju ilmu tahu teknologi dokter layan sel punca sel manfaat upaya layan sehat menteri sehat tetap atur menteri sehat nomor 32 2018 selenggara layan sel punca danatau sel tuju...,Klikdokter
3,stem cell tumbuh manusia beda jelas,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/stem-cell-tumbuhan-dan-manusia-itu-berbeda-ini-penjelasannya,19 April 2025,tumbuh manusia milik stem cell nnamun milik beda struktur fungsi aplikasi penasaran yuk ajar ulas,tumbuh manusia milik stem cell nnamun milik beda struktur fungsi aplikasi penasaran yuk ajar ulas stem cell stem cell sel milik fungsi khusus belah diferensiasi sel milik fungsi spesifik stem cell kenal sebut sel dasar sel punca sel induk manusia stem cell kembang otot saraf darah proses sembuh regenerasi jaring stem cell tumbuh sel meristem milik fungsi khusus diferensiasi stem cell manusia sel induk ujung akar batang jaring kambium sel tumbuh tumbuh baik artikel beda contoh terap exosome s...,Klikdokter
4,agam model victorias secret fashion show kenal bentuk tubuh perempuan,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-kesehatan/mengenal-keberagaman-bentuk-tubuh-perempuan,17 Oktober 2024,victorias secret ubah standar cantik show baru raya agam bentuk tubuh warna kulit ukur pesan kuat selflove body positivity,victorias secret ubah standar cantik show baru raya agam bentuk tubuh warna kulit ukur pesan kuat selflove body positivity tanggal 15 oktober 2024 victorias secret gelar fashion show tahun dinantinanti tampil koleksi baru suasana beda banding tahuntahun usaha dulunya kenal standar ketat model tubuh langsing ubah arah kenal agam bentuk tubuh ajang gengsi show baru panggung victorias secret ramai model dari bentuk tubuh ukur warna kulit tanda langkah jalan dunia fashion inklusivitas victorias ...,Klikdokter


# Halodoc

In [37]:
halodoc_df = pd.read_csv('../artikel/halodoc.csv')
halodoc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        72 non-null     object
 1   tag          72 non-null     object
 2   link         72 non-null     object
 3   date         69 non-null     object
 4   description  72 non-null     object
 5   content      69 non-null     object
dtypes: object(6)
memory usage: 3.5+ KB


In [38]:
halodoc_df['source'] = 'halodoc'

In [39]:
halodoc_df.isnull().sum()

title          0
tag            0
link           0
date           3
description    0
content        3
source         0
dtype: int64

In [40]:
halodoc_df['title'].duplicated().sum()

np.int64(0)

In [41]:
halodoc_df.dropna(inplace=True)

In [42]:
halodoc_df.head()

,title,tag,link,date,description,content,source
0,Kenalan dengan Manfaat Ceramide untuk Perawatan Kulit,"Perawatan Kulit, Kecantikan",https://www.halodoc.com/artikel/kenalan-dengan-manfaat-ceramide-untuk-perawatan-kulit,15 September 2025,Ceramide mampu melindungi skin barrier secara efektif.,Advertisement\nTemukan lebih banyak\nHalodoc\nPerawatan Kulit\nmelembapkan\ntabir surya\nLotion\nLosion\nPerawatan\nCeradan\nlotion\nskincare\nMoisturizer\nSkincare\nhalodoc\nPerawatan kulit\nBeli vitamin dan suplemen\nHALODOC\nlosion\npelembap\nTabir surya\nmoisturizer\nMoisturizing\nPelembap\nperawatan kulit\nKenalan dengan Manfaat Ceramide untuk Perawatan Kulit\nKecantikan\nPerawatan Kulit\n7 menit\nDitinjau oleh \ndr. Erlian Dimas SpDVE\n 15 September 2025\nCeramide mampu melindungi ski...,halodoc
1,Sakit Perut Sebelah Kanan Bawah Sampai ke Pinggang: Penyebab dan Cara Mengatasinya,Sakit Pinggang,https://www.halodoc.com/artikel/sakit-perut-sebelah-kanan-bawah-sampai-ke-pinggang-penyebab-dan-cara-mengatasinya,15 September 2025,"Sakit perut sebelah kanan bawah sampai ke pinggang bisa jadi tanda masalah serius, kenali penyebabnya.","Advertisement\nSakit Perut Sebelah Kanan Bawah Sampai ke Pinggang: Penyebab dan Cara Mengatasinya\nSakit Pinggang\n5 menit\nDitinjau oleh \ndr. Fauzan Azhari SpPD\n 15 September 2025\nSakit perut sebelah kanan bawah sampai ke pinggang bisa jadi tanda masalah serius, kenali penyebabnya.\nDAFTAR ISI\nPenyebab Sakit Perut Sebelah Kanan Bawah Sampai ke Pinggang\nDiagnosis dan Pemeriksaan\nCara Mengatasi Sakit Perut Sebelah Kanan Bawah Sampai ke Pinggang\nKesimpulan\nFAQ\nSakit perut sebelah kan...",halodoc
2,"Mual dan Muntah, Kenali Gejala Alami Infeksi Astrovirus","Demam, Astrovirus",https://www.halodoc.com/artikel/mual-dan-muntah-kenali-gejala-alami-infeksi-astrovirus,15 September 2025,"Infeksi astrovirus dapat menyebabkan gastroenteritis dengan berbagai gejala seperti mual, muntah, dan diare.","Advertisement\nMual dan Muntah, Kenali Gejala Alami Infeksi Astrovirus\nAstrovirus\nDemam\n4 menit\nDitinjau oleh \ndr. Budiyanto, MARS\n 15 September 2025\nInfeksi astrovirus dapat menyebabkan gastroenteritis dengan berbagai gejala seperti mual, muntah, dan diare.\nDAFTAR ISI\nGejala Infeksi Astrovirus\nMuntah: Gejala Umum Infeksi Astrovirus\nPenyebab Infeksi Astrovirus\nBagaimana Penularan Astrovirus Terjadi?\nDiagnosis Infeksi Astrovirus\nPengobatan Infeksi Astrovirus\nKapan Obat Antimun...",halodoc
3,Apa Saja Kandungan Vitamin yang Terdapat dalam Buah Salak?,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandungan-vitamin-yang-terdapat-dalam-buah-salak,15 September 2025,"Buah salak mengandung nutrisi vitamin A, B, C, zat besi, serta karbohidrat.","Advertisement\nApa Saja Kandungan Vitamin yang Terdapat dalam Buah Salak?\nMakanan Sehat\n7 menit\nDitinjau oleh \ndr. Fauzan Azhari SpPD\n 15 September 2025\nBuah salak mengandung nutrisi vitamin A, B, C, zat besi, serta karbohidrat.\nDaftar Isi:\nKandungan Vitamin dan Nutrisi pada Buah Salak\nManfaat Salak untuk Kesehatan Tubuh\nPotensi Efek Samping Konsumsi Salak\nTips Konsumsi Salak yang Aman dan Sehat\nKesimpulan\nFAQ\nSalak adalah buah tropis yang populer di Indonesia dan dikenal deng...",halodoc
4,Ketahui Fungsi Rongga Mulut dalam Sistem Pencernaan Manusia,Mulut dan Gigi,https://www.halodoc.com/artikel/ketahui-fungsi-rongga-mulut-dalam-sistem-pencernaan-manusia,15 September 2025,"Rongga mulut berfungsi untuk mengunyah, memproses makanan secara kimiawi, dan menyalurkan makanan ke dalam lambung.","Advertisement\nKetahui Fungsi Rongga Mulut dalam Sistem Pencernaan Manusia\nMulut dan Gigi\n7 menit\nDitinjau oleh \ndr. Budiyanto, MARS\n 15 September 2025\nRongga mulut berfungsi untuk mengunyah, memproses makanan secara kimiawi, dan menyalurkan makanan ke dalam lambung.\nDAFTAR ISI\nFungsi Utama Rongga Mulut\nAnatomi dan Fungsi Mulut pada Manusia\nCara Menjaga Fungsi Rongga Mulut\nMasalah Kesehatan yang Umum Terjadi pada Mulut\nRongga mulut adalah bagian awal dari s

In [43]:
def preprocess_content_halodoc(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'^.*?daftar isi', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'mau tahu.*?referensi', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'referensi.*$', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [44]:
halodoc_df['title'] = halodoc_df['title'].apply(lambda x: preprocess_title_desc(x))
halodoc_df['description'] = halodoc_df['description'].apply(lambda x: preprocess_title_desc(x))
halodoc_df['content'] = halodoc_df['content'].apply(lambda x: preprocess_content_halodoc(x))

In [45]:
halodoc_df.head()

,title,tag,link,date,description,content,source
0,kenal manfaat ceramide awat kulit,"Perawatan Kulit, Kecantikan",https://www.halodoc.com/artikel/kenalan-dengan-manfaat-ceramide-untuk-perawatan-kulit,15 September 2025,ceramide lindung skin barrier efektif,manfaat ceramide kulit rekomendasi skincare kandung ceramide produk ceramide hasil optimal efek samping perhati tips pilih produk ceramide dokter ceramide jenis asam lemak nama lipid kandung sel kulit manusia bentuk 50 persen lapis kulit luar epidermis asam lemak kerap jadi salah bahan produk awat kulit kerja lindung halang papar radikal bebas potensi rusak kulit ceramide efektif tahan lapis lindung kulit skin barrier kandung ceramide tingkat hidrasi kulit lebih kenyal lembab radiance by web...,halodoc
1,sakit perut belah kanan sampai pinggang sebab atas,Sakit Pinggang,https://www.halodoc.com/artikel/sakit-perut-sebelah-kanan-bawah-sampai-ke-pinggang-penyebab-dan-cara-mengatasinya,15 September 2025,sakit perut belah kanan sampai pinggang tanda serius kenal sebab,sebab sakit perut belah kanan sampai pinggang diagnosis periksa atas sakit perut belah kanan sampai pinggang simpul faq sakit perut belah kanan sampai pinggang kondisi nyeri muncul perut kanan jalar pinggang sakit sifat ringan berat gantung sebab anatomi perut kanan isi organ usus buntu usus salur kemih wanita ovarium kanan tuba falopi pinggang kanan kait otot tulang ginjal sakit perut belah kanan sampai pinggang asal dari organ beda sebab sakit perut belah kanan sampai pinggang sebab timbul...,halodoc
2,mual muntah kenal gejala alami infeksi astrovirus,"Demam, Astrovirus",https://www.halodoc.com/artikel/mual-dan-muntah-kenali-gejala-alami-infeksi-astrovirus,15 September 2025,infeksi astrovirus sebab gastroenteritis gejala mual muntah diare,gejala infeksi astrovirus muntah gejala infeksi astrovirus sebab infeksi astrovirus tular astrovirus diagnosis infeksi astrovirus obat infeksi astrovirus obat antimuntah komplikasi infeksi astrovirus cegah infeksi astrovirus dokter astrovirus jenis virus rna sebab infeksi salur cerna anakanak infeksi seringkali timbul gejala diare mual muntah demam ringan sakit perut bahaya infeksi astrovirus sebab dehidrasi bayi anak gejala infeksi astrovirus infeksi astrovirus gejala mencret darah mual mun...,halodoc
3,kandung vitamin buah salak,Makanan Sehat,https://www.halodoc.com/artikel/apa-saja-kandungan-vitamin-yang-terdapat-dalam-buah-salak,15 September 2025,buah salak kandung nutrisi vitamin a b c zat besi karbohidrat,kandung vitamin nutrisi buah salak manfaat salak sehat tubuh potensi efek samping konsumsi salak tips konsumsi salak aman sehat simpul faq salak buah tropis populer indonesia kenal kulit sisik rupa kulit ular buah milik manis asam sepat unik lezat salak kaya nutrisi manfaat sehat rutin konsumsi buah sehat kandung vitamin dalam penasaran saa kandung manfaat buah salak sehat tubuh tahu lengkap kandung vitamin nutrisi buah salak salak buah milik kulit rupa sisik manis daging buah asal dari indo...,halodoc
4,tahu fungsi rongga mulut sistem cerna manusia,Mulut dan Gigi,https://www.halodoc.com/artikel/ketahui-fungsi-rongga-mulut-dalam-sistem-pencernaan-manusia,15 September 2025,rongga mulut fungsi kunyah proses makan kimiawi salur makan lambung,fungsi utama rongga mulut anatomi fungsi mulut manusia jaga fungsi rongga mulut sehat mulut rongga mulut dari sistem cerna manusia kali anggap sederhana padaha fungsil mulut milik peran kompleks vital proses cerna makan dari kunyah hasil enzim mulut gerbang utama tentu tubuh proses nutrisi komunikasi pasti lebih sulit rongga mulut tahu fungsi mulut fungsi utama rongga mulut mulut milik fungsi utama tubuh terima makan mulut makan masuk tubuh kecap lidah milik reseptor nikmat makan kunyah maka...,halodoc


# Alodokter

In [46]:
alodokter_df = pd.read_csv('../artikel/alodokter.csv')
alodokter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        100 non-null    object
 1   tag          100 non-null    object
 2   link         100 non-null    object
 3   date         100 non-null    object
 4   description  100 non-null    object
 5   content      100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [47]:
alodokter_df['source'] = 'alodokter'
alodokter_df.head()

,title,tag,link,date,description,content,source
0,"Overthinking, Kenali Ciri, Penyebab, Dampak, dan Cara Mengatasinya",HIDUP SEHAT,https://www.alodokter.com/overthinking-kenali-ciri-penyebab-dampak-dan-cara-mengatasinya,Terakhir diperbarui: 16 September 2025,"Overthinking adalah kebiasaan memikirkan sesuatu secara berlebihan hingga membuat pikiran lelah. Jika terjadi terus-menerus, kondisi...","Overthinking adalah kebiasaan memikirkan sesuatu secara berlebihan hingga membuat pikiran lelah. Jika terjadi terus-menerus, kondisi ini bisa mengganggu aktivitas sehari-hari dan memicu terjadinya stres dan kecemasan.\r\nOverthinking sering disalahartikan sebagai rasa khawatir atau terlalu berhati-hati dalam mengambil keputusan. Padahal, kondisi ini berbeda. Overthinking terjadi ketika seseorang memikirkan suatu hal secara berlebihan, sehingga keputusan kecil terasa berat dan langkah sederha...",alodokter
1,"Trauma Healing, Cara Membantu Diri Pulih dari Luka Batin",KESEHATAN,https://www.alodokter.com/trauma-healing-cara-membantu-diri-pulih-dari-luka-batin,Terakhir diperbarui: 16 September 2025,Trauma healing adalah proses penting untuk membantu seseorang pulih dari luka batin. Pengalaman traumatis sering...,"Trauma healing adalah proses penting untuk membantu seseorang pulih dari luka batin. Pengalaman traumatis sering kali meninggalkan bekas mendalam yang memengaruhi kehidupan sehari-hari, sehingga perlu diatasi dengan tepat.\r\nTrauma bisa muncul dari berbagai peristiwa, seperti bencana alam, kehilangan orang tercinta, hingga kekerasan fisik, seksual, maupun emosional. Jika dibiarkan, trauma dapat menimbulkan dampak jangka panjang, seperti kesulitan bersosialisasi serta berkembang menjadi gang...",alodokter
2,"Mansplaining, Perilaku Merendahkan yang Sering Tak Disadari",KESEHATAN,https://www.alodokter.com/mansplaining-perilaku-merendahkan-yang-sering-tak-disadari,Terakhir diperbarui: 16 September 2025,"Mansplaining adalah pola komunikasi ketika seseorang, biasanya laki-laki, menjelaskan sesuatu dengan cara merendahkan atau menggurui,...","Mansplaining adalah pola komunikasi ketika seseorang, biasanya laki-laki, menjelaskan sesuatu dengan cara merendahkan atau menggurui, terutama kepada wanita. Penting memahami fenomena ini sekaligus mengetahui cara tepat menghadapinya.\r\nMansplaining bisa muncul pada interaksi di media sosial, lingkungan kerja, maupun ruang publik. Sering kali hal tersebut terjadi tanpa disadari, bahkan oleh pelakunya sendiri, karena dipengaruhi budaya patriarki, stereotipe gender, atau kurangnya kesadaran u...",alodokter
3,"Fungsi Otak Kiri, Kunci Kemampuan Analitis dan Matematika",KESEHATAN,https://www.alodokter.com/fungsi-otak-kiri-kunci-kemampuan-analitis-dan-matematika,Terakhir diperbarui: 16 September 2025,"Fungsi otak kiri memegang peranan penting dalam kemampuan berpikir logis, berbahasa, dan memecahkan masalah sehari-hari....","Fungsi otak kiri memegang peranan penting dalam kemampuan berpikir logis, berbahasa, dan memecahkan masalah sehari-hari. Dengan memahami fungsi otak kiri, Anda bisa mengasah kelebihan diri dan membantu perkembangan anak sejak dini.\r\nSetiap bagian otak manusia memiliki fungsi masing-masing yang saling melengkapi. Jika fungsi otak kanan lebih dominan dalam hal kreativitas, imajinasi, dan intuisi, maka fungsi otak kiri berperan penting dalam kemampuan analisis, logika, bahasa, dan komunikasi....",alodokter
4,"Daun Jati Belanda, Kenali Manfaatnya untuk Kesehatan",KESEHATAN,https://www.alodokter.com/daun-jati-belanda-kenali-manfaatnya-untuk-kesehatan,Terakhir diperbarui: 16 September 2025,Daun jati belanda merupakan herbal tradisional yang dipercaya membantu menurunkan berat badan dan menjaga pencernaan....,"Daun jati belanda merupakan herbal tradisional yang dipercaya membantu menurunkan berat badan dan menjaga pencernaan. Di Indonesia, tanaman ini kian populer di kalangan mereka yang mencari cara diet alami tanpa efek samping serius.\r\nMasyarakat Indonesia banyak mengenal daun jati be

In [48]:
alodokter_df['title'].duplicated().sum()

np.int64(34)

In [49]:
alodokter_df.drop_duplicates(subset='title', inplace=True)
alodokter_df['title'].duplicated().sum()

np.int64(0)

In [50]:
def preprocess_content_alodokter(text):
    text = str(text).lower()
    text = normalize_text(text, istilah_kesehatan)  # Normalisasi istilah kesehatan
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Hapus URL
    text = re.sub(r'\@w+|\#', '', text)  # Hapus mention & hashtag
    text = re.sub(r'advertisement', '', text, flags=re.IGNORECASE)  # Hapus "advertisement"
    text = re.sub(r'^.*?daftar isi', '', text, flags=re.DOTALL|re.IGNORECASE)  # Hapus "daftar isi"
    text = re.sub(r'referensi.*$', '', text, flags=re.DOTALL|re.IGNORECASE)  # Hapus "referensi"
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text)  # Normalisasi tanggal
    text = re.sub(r'\s+', ' ', text)  # Hapus spasi berlebih
    text = re.sub(r'[^\w\s]', '', text)  # Hapus karakter non-alfanumerik
    text = ''.join([c for c in text if c not in string.punctuation])  # Hapus tanda baca
    text = ' '.join([w for w in text.split() if w not in custom_stop_words])  # Stopword removal
    text = ' '.join(stem_tokens(text.split()))  # Stemming
    return text.strip()

In [51]:
def preprocess_date_alodokter(text):
    text = str(text).replace("Terakhir diperbarui:", "").strip()
    try:
        # Set locale ke Indonesia
        locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
        dt = datetime.strptime(text, "%d %B %Y")
        return dt.strftime("%d %B %Y")
    except Exception:
        raise ValueError(f"Format tanggal tidak dikenali: {text}")

In [52]:
alodokter_df['title'] = alodokter_df['title'].apply(lambda x: preprocess_title_desc(x))
alodokter_df['description'] = alodokter_df['description'].apply(lambda x: preprocess_title_desc(x))
alodokter_df['content'] = alodokter_df['content'].apply(lambda x: preprocess_content_alodokter(x))
alodokter_df['date'] = alodokter_df['date'].apply(lambda x: preprocess_date_alodokter(x))
alodokter_df['tag'] = alodokter_df['tag'].str.lower()

In [53]:
alodokter_df.head()

,title,tag,link,date,description,content,source
0,overthinking kenal ciri sebab dampak atas,hidup sehat,https://www.alodokter.com/overthinking-kenali-ciri-penyebab-dampak-dan-cara-mengatasinya,16 September 2025,overthinking biasa pikir pikir lelah terusmenerus kondisi,overthinking biasa pikir pikir lelah terusmenerus kondisi ganggu aktivitas seharihari picu stres cemas overthinking disalahartikan khawatir berhatihati ambil putus kondisi beda overthinking pikir putus berat langkah sederhana sulit ambil overthinking abai kondisi bawa dampak buruk sehat mental kualitas hidup derita nali ciriciri sebab atas kondisi berlarutlarut ciriciri overthinking nali overthinker langsung sadar milik sifat overthinking kenal ciriciri overthinking kondisi ciriciri overthin...,alodokter
1,trauma healing bantu pulih dari luka batin,kesehatan,https://www.alodokter.com/trauma-healing-cara-membantu-diri-pulih-dari-luka-batin,16 September 2025,trauma healing proses bantu pulih dari luka batin alam traumatis,trauma healing proses bantu pulih dari luka batin alam traumatis kali tinggal bekas dalam pengaruh hidup seharihari atas trauma muncul dari peristiwa bencana alam hilang orang cinta keras fisik seksual emosional biar trauma timbul dampak jangka sulit sosial kembang ganggu cemas depresi ptsd kondisi paham proses trauma healing pulih tahap proses kerap kait fase duka alami hadap hilang trauma healing trauma healing tuju bantu sintas kurang dampak negatif alam traumatis bangun aman langkah deka...,alodokter
2,mansplaining perilaku rendah sadar,kesehatan,https://www.alodokter.com/mansplaining-perilaku-merendahkan-yang-sering-tak-disadari,16 September 2025,mansplaining pola komunikasi lakilaki rendah guru,mansplaining pola komunikasi lakilaki rendah guru wanita paham fenomena hadap mansplaining muncul interaksi media sosial lingkung kerja ruang publik kali sadar laku pengaruh budaya patriark stereotipe gender kurang sadar komunikasi tara ciriciri mansplaining nali komunikasi dari pria wanita mansplaining istilah acu situasi nada rendah anggap lawan bicara kurang paham erti lebih ahli lebih mudah nali perilaku mansplaining tanda muncul ulang lawan bicara paham topik koreksi lurus jelas orang a...,alodokter
3,fungsi otak kiri kunci mampu analitis matematika,kesehatan,https://www.alodokter.com/fungsi-otak-kiri-kunci-kemampuan-analitis-dan-matematika,16 September 2025,fungsi otak kiri pegang peran mampu pikir logis bahasa pecah seharihari,fungsi otak kiri pegang peran mampu pikir logis bahasa pecah seharihari paham fungsi otak kiri asah lebih bantu kembang anak otak manusia milik fungsi masingmasing lengkap fungsi otak kanan lebih dominan kreativitas imajinasi intuisi fungsi otak kiri peran mampu analisis logika bahasa komunikasi kabar baik mampu otak kiri latih optimal dukung aktivitas seharihari fungsi otak kiri paham otak kiri bantu optimal ajar ambil putus seharihari fungsi otak kiri 1 pemrosesan bahasa otak kiri atur mam...,alodokter
4,daun jati belanda kenal manfaat sehat,kesehatan,https://www.alodokter.com/daun-jati-belanda-kenali-manfaatnya-untuk-kesehatan,16 September 2025,daun jati belanda herbal tradisional percaya bantu turun berat badan jaga cerna,daun jati belanda herbal tradisional percaya bantu turun berat badan jaga cerna indonesia tanam kian populer kalang cari diet alami efek samping serius masyarakat indonesia kenal daun jati belanda bahan utama jamu langsing teh herbal popularitas lepas dari percaya turuntemurun daun manfaat sehat jaga berat badan heran daun jati belanda pilih dari gaya hidup sehat daun jati belanda manfaat sehat daun jati belanda kandung serat flavonoid tanin saponin antioksidan alami peran jaga sehat tubuh k...,alodokter


# Detik Health

In [54]:
detik_df = pd.read_csv('../artikel/detik_health.csv', sep=';')
detik_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        100 non-null    object 
 1   tag          0 non-null      float64
 2   link         100 non-null    object 
 3   date         100 non-null    object 
 4   description  100 non-null    object 
 5   content      100 non-null    object 
dtypes: float64(1), object(5)
memory usage: 4.8+ KB


In [55]:
detik_df['source'] = 'detik'

In [56]:
detik_df['tag'] = detik_df['tag'].fillna('Kesehatan Umum')

In [57]:
detik_df['title'].duplicated().sum()

np.int64(50)

In [58]:
detik_df.drop_duplicates(subset='title', inplace=True)
detik_df['title'].duplicated().sum()

np.int64(0)

In [59]:
detik_df.head()

,title,tag,link,date,description,content,source
0,"Klaim BPJS Kesehatan untuk Gangguan Jiwa Naik! Hampir Rp 2 Triliun, Ini Rinciannya",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8115094/klaim-bpjs-kesehatan-untuk-gangguan-jiwa-naik-hampir-rp-2-triliun-ini-rinciannya,2025-09-16 10:22:44,Direktur Utama BPJS Kesehatan Ali Ghufron Mukti mengungkapkan bahwa nilai klaim program Jaminan Kesehatan Nasional (JKN) untuk masalah kesehatan mental mencapai Rp 6 triliun lebih.,"Direktur Utama BPJS Kesehatan Ali Ghufron Mukti mengungkapkan bahwa nilai klaim program Jaminan Kesehatan Nasional (JKN) untuk masalah kesehatan mental mencapai Rp 6 triliun lebih. Jumlah tersebut berdasarkan data nilai pengeluaran total periode tahun 2020-2024. Ghufron menyebut jumlah biaya yang dikeluarkan untuk menangani masalah kesehatan mental naik setiap tahun. ""Biayanya dalam waktu 2020-2024, sekitar Rp 6,7 triliun. Biaya untuk merawat yang sudah dibayar oleh BPJS untuk perawatan oran...",detik
1,"Catat Rekor Baru, Pria Ini Positif COVID-19 Selama 2 Tahun!",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114957/catat-rekor-baru-pria-ini-positif-covid-19-selama-2-tahun,2025-09-16 09:22:47,Seorang pria dengan sistem kekebalan tubuh yang lemah mengidap COVID-19 akut yang berkelanjutan selama lebih dari 750 hari.,"Seorang pria dengan sistem kekebalan tubuh yang lemah mengidap COVID-19 akut yang berkelanjutan selama lebih dari 750 hari. Selama periode ini, ia mengalami gejala pernapasan yang persisten dan dirawat di rumah sakit sebanyak lima kali. Meski durasinya panjang, kondisi pria ini berbeda dengan long COVID. Sebab, gejalanya bukanlah gejala yang menetap setelah virus menghilang, melainkan fase virus SARS-CoV-2 yang berlanjut selama lebih dari dua tahun. Kondisi ini mungkin hanya terjadi pada ora...",detik
2,Strategi BPOM RI Berantas Produk Obat dan Makanan Mengandung Bahan Berbahaya,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114838/strategi-bpom-ri-berantas-produk-obat-dan-makanan-mengandung-bahan-berbahaya,2025-09-16 09:22:49,Badan Pengawas Obat dan Makanan (BPOM RI) bergerak cepat untuk memberantas produk obat dan makanan mengandung bahan berbahaya.,"Badan Pengawas Obat dan Makanan (BPOM RI) bergerak cepat untuk memberantas produk obat dan makanan mengandung bahan berbahaya. BPOM berkomitmen untuk menyelesaikan permasalahan ini dari hulu ke hilir melalui Kick Off Aksi Bersama Pencegahan dan Penanganan Rantai Pasok Bahan Berbahaya/Bahan Dilarang Deputi Bidang Penindakan BPOM, Tubagus Ade Hidayat menyampaikan bahwa tren tindak pidana terkait obat dan makanan terus meningkat dalam 5 tahun terakhir. Pada 2024, PPNS BPOM menangani 282 perkara...",detik
3,"Curhat Pria Lahir dari Donor Sperma, Bingung Cari Ayah Kandung",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114689/curhat-pria-lahir-dari-donor-sperma-bingung-cari-ayah-kandung,2025-09-16 08:22:52,Seorang pria di Inggris merupakan salah satu anak yang lahir dari donor sperma.,"Seorang pria di Inggris merupakan salah satu anak yang lahir dari donor sperma. Ia dikandung melalui program bayi tabung atau In Vitro Fertilization (IVF) dengan donor sperma para tahun 2002. Sampai di usia 18 tahun, pria bernama Nat Allbut itu mengajukan permohonan kepada Human Fertilisation and Embryology Authority (HFEA) atau otoritas fertilisasi dan embriologi manusia untuk mendapatkan informasi tentang orang tua kandungnya. Allbut menerima berkas berisi detail tentang donor spermanya, t...",detik
4,"Pastikan Indomie Aman, Indofood Soroti Beda Regulasi RI Vs Taiwan soal Etilen Oksida",Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114504/pastikan-indomie-aman-indofood-soroti-beda-regulasi-ri-vs-taiwan-soal-etilen-oksida,2025-09-16 06:22:54,Laporan kandungan etilen oksida (EtO) dalam Indomie varian Soto Banjar Limau Kuit belakangan memicu kekhawatiran publik.,"Laporan kandungan etilen oksida (EtO) dalam Indomie varian Soto Banjar Limau Kuit belakangan

In [60]:
def preprocess_date_detik(text):
    text = text.strip().strip("'")
    
    dt = None
    try:
        dt = datetime.strptime(text, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        pass
    
    if dt is None:
        try:
            # set locale ke Indonesia
            locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
            dt = datetime.strptime(text, "%A, %d %b %Y %H:%M")
        except Exception:
            raise ValueError(f"Format tanggal tidak dikenali: {text}")
    
    return dt.strftime("%d %B %Y")

In [61]:
def preprocess_content_detik(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'advertisement scroll to continue with content', '', text, flags=re.IGNORECASE) # Remove advertisement
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [62]:
detik_df['title'] = detik_df['title'].apply(lambda x: preprocess_title_desc(x))
detik_df['description'] = detik_df['description'].apply(lambda x: preprocess_title_desc(x))
detik_df['content'] = detik_df['content'].apply(lambda x: preprocess_content_detik(x))
detik_df['date'] = detik_df['date'].apply(lambda x: preprocess_date_detik(x))

In [63]:
detik_df.head()

,title,tag,link,date,description,content,source
0,klaim bpjs sehat ganggu jiwa rp 2 triliun rinciannya,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8115094/klaim-bpjs-kesehatan-untuk-gangguan-jiwa-naik-hampir-rp-2-triliun-ini-rinciannya,16 September 2025,direktur utama bpjs sehat ali ghufron mukti nilai klaim program jamin sehat nasional jkn sehat mental capai rp 6 triliun lebih,direktur utama bpjs sehat ali ghufron mukti nilai klaim program jamin sehat nasional jkn sehat mental capai rp 6 triliun lebih dasar data nilai keluar total periode 2020 sampai 2024 ghufron sebut biaya keluar tangan sehat mental biaya 2020 sampai 2024 rp 67 triliun biaya rawat bayar bpjs awat orang ganggu jiwa odgj ghufron temu awak media surakarta jawa selasa date 2024 biaya keluar bpjs sehat capai rp 19 triliun tingkat dari 2023 rp 16 triliun 2022 rp 12 triliun 2021 rp 1 triliun 2020 rp 93...,detik
1,catat rekor pria positif covid19 2,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114957/catat-rekor-baru-pria-ini-positif-covid-19-selama-2-tahun,16 September 2025,pria sistem kebal tubuh lemah idap covid19 akut lanjut lebih dari 750 hari,pria sistem kebal tubuh lemah idap covid19 akut lanjut lebih dari 750 hari periode alami gejala napas persisten rawat rumah sakit kali durasi kondisi pria beda long covid gejala gejala tetap virus hilang fase virus sarscov2 lanjut lebih dari kondisi orang rentan ahli amerika serikat ingat studi infeksi jangka virus eksplorasi infeksi sel lebih efisien studi tambah bukti varian lebih mudah tular muncul dari infeksi terang ahli epidemiologi universitas harvard william hanage kutip darisciencea...,detik
2,strategi bpom ri berantas produk obat makan kandung bahan bahaya,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114838/strategi-bpom-ri-berantas-produk-obat-dan-makanan-mengandung-bahan-berbahaya,16 September 2025,badan awas obat makan bpom ri gerak cepat berantas produk obat makan kandung bahan bahaya,badan awas obat makan bpom ri gerak cepat berantas produk obat makan kandung bahan bahaya bpom komitmen selesai masalah dari hulu hilir kick off aksi cegah tangan rantai pasok bahan berbahayabahan larang deputi bidang tindak bpom tubagus ade hidayat tren tindak pidana kait obat makan tingkat 5 2024 ppns bpom tangan 282 perkara dari 124 perkara obat napza 55 perkara obat bahan alam 91 perkara kosmetik 12 perkara pangan olah temu nilai ekonomi signifikan perkara obatobat semarang cikarang maru...,detik
3,curhat pria lahir dari donor sperma bingung cari ayah kandung,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114689/curhat-pria-lahir-dari-donor-sperma-bingung-cari-ayah-kandung,16 September 2025,pria inggris salah anak lahir dari donor sperma,pria inggris salah anak lahir dari donor sperma kandung program bayi tabung in vitro fertilization ivf donor sperma 2002 sampai usia 18 pria nama nat allbut aju mohon human fertilisation and embryology authority hfea otoritas fertilisasi embriologi manusia informasi orang tua kandung allbut terima berkas isi detail donor sperma badan warna mata profesi kewarganegaraan ayah kandung hasil tes dna jalan tentang detail dari hfea tahuntahun hidup milik latar keliru kutip dariindependent uk salah ...,detik
4,pasti indomie aman indofood sorot beda regulasi ri vs taiwan etilen oksida,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-8114504/pastikan-indomie-aman-indofood-soroti-beda-regulasi-ri-vs-taiwan-soal-etilen-oksida,16 September 2025,lapor kandung etilen oksida eto indomie varian soto banjar limau kuit picu khawatir publik,lapor kandung etilen oksida eto indomie varian soto banjar limau kuit picu khawatir publik temu lapor otoritas taiwan tolerir kandung eto pangan alias wajib nol deteksi risiko saraf picu kanker batas kadar corporate secretary pt indofood cbp sukses makmur gideon a putro tekan resmi impor langsung varian taiwan varian impor importir distributor resmi persero sampai varian pasar ekspor taiwan gideon

# Kompas Health

In [64]:
kompas_df = pd.read_csv('../artikel/kompas_health.csv', sep=';')
kompas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        51 non-null     object
 1   tag          51 non-null     object
 2   link         51 non-null     object
 3   date         51 non-null     object
 4   description  51 non-null     object
 5   content      51 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [65]:
kompas_df['source'] = 'kompas'

In [66]:
kompas_df['title'].duplicated().sum()

np.int64(34)

In [67]:
kompas_df.drop_duplicates(subset='title', inplace=True)
kompas_df['title'].duplicated().sum()

np.int64(0)

In [68]:
kompas_df.head()

,title,tag,link,date,description,content,source
0,Mengapa Anak Harus Mendapatkan Vaksin Campak? Ini Penjelasan Kemenkes,HEALTH,https://health.kompas.com/read/25I15161500668/mengapa-anak-harus-mendapatkan-vaksin-campak-ini-penjelasan-kemenkes,"15/09/2025, 16:15 WIB",Jangan abaikan vaksin campak. Vaksin ini melindungi anak dari penyakit menular berbahaya. Simak penjelasan terkait manfaatnya.,"KOMPAS.com -Mengapa vaksin campak penting untuk anak? Menurut Kementerian Kesehatan (Kemenkes), vaksi tersebut tidak hanya mencegah penyakit campak, tapi juga melindungi anak dari risiko komplikasi serius yang bisa berujung kematian. ""Untuk memberikan perlindungan yang optimal terhadap penyakit campak maka imunisasi campak diberikan sebanyak tiga kali,"" kata Kepala Biro Informasi dan Komunikasi Publik Kementerian Kesehatan, Aji Muhawarman kepadaAntara, Senin (15/9/2025). Baca juga: Ia menamb...",kompas
1,Cara Sederhana Mengatasi Brain Fog,HEALTH,https://health.kompas.com/read/25I15144420968/cara-sederhana-mengatasi-brain-fog,"15/09/2025, 14:44 WIB","Brain fog merujuk pada sekumpulan gejala kognitif seperti kesulitan berkonsentrasi, mudah lupa, dan kelambatan berpikir.","KOMPAS.com -Ketika kesibukan dan tingkat stres meningkat, bukan hal yang aneh jika kita tiba-tiba masuk ke ruangan dan lupa tujuan, lupa istilah sesuatu padahal sudah ada ""di ujung lidah"", atau kesulitan untuk tetap fokus pada tugas-tugas sederhana. Kekeruhan mental ini sering disebut kabut otak (brain fog), dan meskipun bukan kondisi medis, brain fog merujuk pada sekumpulan gejala kognitif seperti kesulitan berkonsentrasi, mudah lupa, dan kelambatan berpikir. Serangan kabut otak cukup umum,...",kompas
2,Pemprov Babel Segera Bangun Rumah Sakit Jantung dan Stroke,HEALTH,https://health.kompas.com/read/25I14224517668/pemprov-babel-segera-bangun-rumah-sakit-jantung-dan-stroke,"14/09/2025, 22:45 WIB","Pemprov Kepulauan Babel rencanakan pembangunan RS Jantung dan Stroke dekat Bandara, untuk meningkatkan layanan kesehatan masyarakat.","KOMPAS.com- Pembangunan Rumah Sakit Jantung dan Stroke di Provinsi Kepulauan Bangka Belitung (Babel) bertujuan memperkuat infrastruktur layanan kesehatan, sekaligus menjawab kebutuhan masyarakat penderita jantung dan stroke agar tidak perlu berobat ke luar daerah. ""Insya Allah dalam waktu dekat ini, kita realisasikan pembangunan Rumah Sakit Jantung dan Stroke,"" kata Gubernur Kepulauan Babel Hidayat Arsani di Pangkalpinang, seperti dikutip dari Antara, Minggu (14/9). Ia mengatakan, kehadiran ...",kompas
3,"Kusta Jadi Masalah Serius di Papua Barat, Kemenkes Diminta Bentuk Tim Investigasi Penanganan Kasus",HEALTH,https://health.kompas.com/read/25I14222312268/kusta-jadi-masalah-serius-di-papua-barat-kemenkes-diminta-bentuk-tim-investigasi,"14/09/2025, 22:23 WIB","DPD RI desak Kemenkes bentuk tim investigasi kusta di Papua Barat. Penanganan masih terkendala obat, tenaga medis, dan tingginya risiko penularan.","KOMPAS.com- Kasus kusta di Papua Barat masih menjadi persoalan serius. Hingga tahun 2024, tercatat 796 orang penderita tersebar di enam kabupaten, dengan prevalensi kasus mencapai 13,76 per 10 ribu penduduk. Penyakit menular yang disebabkan bakteri mycobacterium leprae ini sangat memerlukan penanganan komprehensif. Ketua Komite III DPD Republik Indonesia Filep Wamafma meminta Kementerian Kesehatan (Kemenkes) membentuk tim investigasi untuk penanganan kasus tersebut di Provinsi Papua Barat. B...",kompas
4,Modul Penanganan Psikosomatis Dokter Indonesia Dipresentasikan di Jerman,HEALTH,https://health.kompas.com/read/25I13184500268/modul-penanganan-psikosomatis-dokter-indonesia-dipresentasikan-di-jerman,"13/09/2025, 18:45 WIB","Sejak 2010, dr. Andri memfokuskan praktik klinisnya pada pasien dengan gangguan psikosomatis dan kecemasan.","KOMPAS.com -Dalam ajang bergengsi European Academy of Psychosomatic Medicine (EAPM) Annual Congress 2025 yang berlangsung pada 11–13 September di Munich, dr. Andri, Sp.KJ, FAPM, kembali mewakili Indonesia dengan mem

In [69]:
def preprocess_date_kompas(text):
    text = text.strip().replace(" WIB", "")  # hapus WIB
    
    try:
        dt = datetime.strptime(text, "%d/%m/%Y, %H:%M")
    except ValueError:
        raise ValueError(f"Format tanggal tidak dikenali: {text}")
    
    return dt.strftime("%d %B %Y")

In [70]:
def preprocess_content_kompas(text):
    text = text.lower()
    text = normalize_text(text, istilah_kesehatan)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'kompas\.com\s*[–-]\s*', '', text, flags=re.MULTILINE) # Remove [NAMA KOTA], kompas.com -
    text = re.sub(r'baca juga:.*', '', text, flags=re.MULTILINE) # Remove baca juga
    text = re.sub(r'^.*\nartikel kompas\.id$', '', text, flags=re.MULTILINE) # Remove artikel kompas.id
    text = re.sub(r'\s*\d{1,2}[/-]\d{1,2}(?:[/-]\d{2,4})?\s*', 'DATE', text) # Remove dates in parentheses
    text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
    text = re.sub(r'[^\w\s]', '', text) # Remove non-words and non-white spaces
    text = ''.join([char for char in text if char not in string.punctuation]) # Remove punctuations
    text = ' '.join([word for word in text.split() if word not in custom_stop_words]) # Stopwords removal
    text = ' '.join(stem_tokens(text.split()))

    return text

In [71]:
kompas_df['title'] = kompas_df['title'].apply(lambda x: preprocess_title_desc(x))
kompas_df['description'] = kompas_df['description'].apply(lambda x: preprocess_title_desc(x))
kompas_df['content'] = kompas_df['content'].apply(lambda x: preprocess_content_kompas(x))
kompas_df['date'] = kompas_df['date'].apply(lambda x: preprocess_date_kompas(x))

In [72]:
kompas_df.head()

,title,tag,link,date,description,content,source
0,anak vaksin campak jelas kemenkes,HEALTH,https://health.kompas.com/read/25I15161500668/mengapa-anak-harus-mendapatkan-vaksin-campak-ini-penjelasan-kemenkes,15 September 2025,abai vaksin campak vaksin lindung anak dari sakit tular bahaya simak jelas kait manfaat,vaksin campak anak menteri sehat kemenkes vaksi cegah sakit campak lindung anak dari risiko komplikasi serius ujung mati lindung optimal sakit campak imunisasi campak kali kepala biro informasi komunikasi publik menteri sehat aji muhawarman kepadaantara senin date,kompas
1,sederhana atas brain fog,HEALTH,https://health.kompas.com/read/25I15144420968/cara-sederhana-mengatasi-brain-fog,15 September 2025,brain fog rujuk kumpul gejala kognitif sulit konsentrasi mudah lupa lambat pikir,sibuk tingkat stres tingkat aneh tibatiba masuk ruang lupa tuju lupa istilah ujung lidah sulit fokus tugastugas sederhana keruh mental kabut otak brain fog kondisi medis brain fog rujuk kumpul gejala kognitif sulit konsentrasi mudah lupa lambat pikir serang kabut otak iring tambah usia studi 28 persen orang dewasa lapor alami gejala sehat otak akibat langsung dari sehat tubuh alexander billioux md kepala tugas medis program perintah united healthcare,kompas
2,pemprov babel bangun rumah sakit jantung stroke,HEALTH,https://health.kompas.com/read/25I14224517668/pemprov-babel-segera-bangun-rumah-sakit-jantung-dan-stroke,14 September 2025,pemprov pulau babel rencana bangun rs jantung stroke bandara tingkat layan sehat masyarakat,bangun rumah sakit jantung stroke provinsi pulau bangka belitung babel tuju kuat infrastruktur layan sehat butuh masyarakat derita jantung stroke obat daerah insya allah realisasi bangun rumah sakit jantung stroke gubernur pulau babel hidayat arsani pangkalpinang kutip dari minggu date hadir rumah sakit jantung stroke provinsi pulau babel dukung transformasi bidang sehat prioritas nasional,kompas
3,kusta serius papua barat kemenkes bentuk tim investigasi tangan,HEALTH,https://health.kompas.com/read/25I14222312268/kusta-jadi-masalah-serius-di-papua-barat-kemenkes-diminta-bentuk-tim-investigasi,14 September 2025,dpd ri desak kemenkes bentuk tim investigasi kusta papua barat tangan kendala obat tenaga medis tinggi risiko tular,kusta papua barat serius 2024 catat 796 orang derita sebar enam kabupaten prevalensi capai 1376 10 ribu duduk sakit tular sebab bakteri mycobacterium leprae tangan komprehensif ketua komite iii dpd republik indonesia filep wamafma menteri sehat kemenkes bentuk tim investigasi tangan provinsi papua barat,kompas
4,modul tangan psikosomatis dokter indonesia presentasi jerman,HEALTH,https://health.kompas.com/read/25I13184500268/modul-penanganan-psikosomatis-dokter-indonesia-dipresentasikan-di-jerman,13 September 2025,2010 dr andri fokus praktik klinis pasien ganggu psikosomatis cemas,ajang gengsi european academy of psychosomatic medicine eapm annual congress 2025 11 sampai 13 september munich dr andri spkj fapm wakil indonesia bawa presentasi ilmiah dasar alam klinis 17 tangan pasien psikosomatis 2010 dr andri fokus praktik klinis pasien ganggu psikosomatis cemas praktik klinis seharihari fokus psikosomatik jumpa pasien keluh fisik ganggu jantung debar nyeri lambung migrain akar sumber dari kondisi psikis,kompas


# Merging

In [73]:
# Menggabungkan semua dataframe
all_df = pd.concat([biofarma_df, klikdokter_df, halodoc_df, alodokter_df, detik_df, kompas_df], ignore_index=True)
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        420 non-null    object
 1   tag          420 non-null    object
 2   link         420 non-null    object
 3   date         420 non-null    object
 4   description  420 non-null    object
 5   content      420 non-null    object
 6   source       420 non-null    object
dtypes: object(7)
memory usage: 23.1+ KB


In [74]:
all_df.isnull().sum()

title          0
tag            0
link           0
date           0
description    0
content        0
source         0
dtype: int64

In [75]:
all_df.insert(0, 'article_index', range(1, len(all_df) + 1))
all_df.head()

,article_index,title,tag,link,date,description,content,source
0,1,5 rawat ginjal sehat cegah sakit ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/5-cara-merawat-ginjal-agar-sehat-cegah-penyakit-ginjal,3 September 2025,kenal rawat ginjal hindar dari gejala ginjal masalah ajar ciri sakit ginjal cegah sakit ginjal,ginjal organ vital fungsi nyaring darah buang limbah jaga imbang cair tubuh jaga fungsi ginjal turun timbul serius sakit ginjal kembang gejala tahap jaga sehat ginjal 5 rawat ginjal sederhana efektif cegah gejala ginjal masalah 1 minum air putih asupan cair bantu ginjal optimal buang racun sisa metabolisme kurang minum air sebab dehidrasi tingkat risiko batu ginjal biasa minum 8 sampai 10 gelas air putih hari jaga fungsi ginjal sehat 2 batas konsumsi garam asupan garam lebih tingkat tekan da...,Biofarma
1,2,7 tips jaga sehat mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/7-tips-untuk-menjaga-kesehatan-mata,29 Agustus 2025,tips jaga sehat mata nutrisi gizi kacamata lindung olahraga periksa rutin dokter,jaga sehat mata mata peran aktivitas seharihari dari ajar interaksi lingkung sehat mata abai muncul dari iritasi ringan infeksi sakit serius risiko sebab buta jaga mata periksa rutin dokter biasa pola hidup sehat dukung fungsi lihat optimal iring tambah usia biasa seharihari kurang sehat fungsi mata turun cegah langkah sederhana jaga asupan nutrisi istirahat tatap layar lindung mata dari papar sinar lebih jaga kualitas lihat tahan lebih tips jaga sehat mata mata salah organ aktif hari lihat ...,Biofarma
2,3,sepele 10 sebab campak abai,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/jangan-sepelekan-10-penyebab-campak-yang-sering-terabaikan,28 Agustus 2025,kenal faktor utama sebab campak gejala khas cegah imunisasi mmr,sebab campak campak sakit tular sebab virus campak tampek sebar cepat dari orang orang mekanisme utama tular percik air liur droplet dari mulut hidung derita batuk bersin bicara tular sentuh benda kontaminasi virus sentuh hidung mulut mata puluh faktor sebab pengaruh vaksinasi campak mmr campak serang individu vaksin vaksin mmr measles mumps rubella bukti efektif cegah sakit anakanak terima vaksin lengkap milik risiko lebih infeksi milik kebal tubuh virus campak tampek virus usia bayi muda v...,Biofarma
3,4,campak sebab gejala cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/campak-penyebab-gejala-dan-pencegahannya,25 Agustus 2025,campak bahaya biar periksa dokter gejala muncul tangan lebih,campak campak rubella sakit tular sebab virus dari keluarga paramyxoviridae genus morbillivirus sakit serang salur napas timbul gejala sistemik demam batuk pilek sakit tenggorok mata merah ruam kulit campak sakit tular pindah dari orang orang udara percik air liur benda kontaminasi virus gejala campak gejala campak muncul inkubasi 7 sampai 14 hari gejala flu demam lemas batuk kering pilek sakit tenggorok diare muntah mata merah air sensitif cahaya bercak putih bintik koplik muncul mulut hari...,Biofarma
4,5,mudah jaga sehat gigi mulut hindar dari sakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/detail/cara-mudah-menjaga-kesehatan-gigi-dan-mulut-agar-terhindar-dari-penyakit,20 Agustus 2025,sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran,sehat gigi mulut sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran proses cerna makan masuk tinggal mulut bersih sisa kembang bakteri sehat mulut kait indah senyum napas segar tentu sehat tubuh akibat jaga sehat gigi mulut abai bersih mulut gigi picu ganggu sehat muncul bau mulut halitosis sisa makan busuk mulut akibat biasa rokok gigi lubang akibat tumpuk plak kikis email gigi radang gusi gingivitis atas kembang ...,Biofarma


In [76]:
all_df.to_csv('../artikel/all_health_articles.csv', index=False)

# Merge Raw Data

In [84]:
alodokter = pd.read_csv('../artikel/alodokter.csv')
biofarma = pd.read_csv('../artikel/biofarma.csv')
klikdokter = pd.read_csv('../artikel/klikdokter.csv')
halodoc = pd.read_csv('../artikel/halodoc.csv')
detik = pd.read_csv('../artikel/detik_health.csv', sep=';')
kompas = pd.read_csv('../artikel/kompas_health.csv', sep=';')

In [85]:
alodokter['date'] = alodokter['date'].apply(lambda x: preprocess_date_alodokter(x))
biofarma['date'] = biofarma['date'].apply(lambda x: preprocess_date_biofarma(x))
klikdokter['date'] = klikdokter['date'].apply(lambda x: preprocess_date_klikdokter(x, bulan))
detik['date'] = detik['date'].apply(lambda x: preprocess_date_detik(x))
kompas['date'] = kompas['date'].apply(lambda x: preprocess_date_kompas(x))

In [86]:
biofarma['source'] = 'Biofarma'
biofarma['tag'] = biofarma['tag'].fillna('Kesehatan Umum')

klikdokter['source'] = 'Klikdokter'

halodoc['source'] = 'halodoc'

alodokter['source'] = 'alodokter'

detik['source'] = 'detik'
detik['tag'] = detik['tag'].fillna('Kesehatan Umum')

kompas['source'] = 'kompas'

In [95]:
halodoc.dropna(inplace=True)

In [96]:
raw_df = pd.concat([biofarma, klikdokter, halodoc, alodokter, detik, kompas], ignore_index=True)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538 entries, 0 to 537
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        538 non-null    object
 1   tag          538 non-null    object
 2   link         538 non-null    object
 3   date         538 non-null    object
 4   description  531 non-null    object
 5   content      538 non-null    object
 6   source       538 non-null    object
dtypes: object(7)
memory usage: 29.6+ KB


In [97]:
raw_df.drop_duplicates(subset='title', inplace=True)

In [98]:
len(raw_df)

420

In [99]:
raw_df.insert(0, 'article_index', range(1, len(raw_df) + 1))
# raw_df.head()

In [100]:
raw_df.to_csv('../artikel/raw_health_articles.csv', index=False)